In [1]:
# prediction based on the geolat and geolon information

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter

train_data = pd.read_csv('dm_all.csv')

# delete all data with empty tags from test data
train_data.dropna(subset=['photoTags'], inplace=True)
# clean all geolat so the values are in the same format
train_data['photoTags'] = train_data['photoTags'].str.replace(":", '')
train_data['photoTags'] = train_data['photoTags'].str.replace("=", '')
#take tags from the column 'photoTags'
train_geolat = train_data[train_data['photoTags'].str.contains('geolat')]
print(len(train_geolat))
k = len(train_geolat)


# we will create excel for all the data that contains geolat information 
train_geolat.to_csv("all_geo_info.csv", encoding="utf-8",index=None)

train_geolat = pd.read_csv('all_geo_info.csv')
list_ln = []
list_ln_true = []
list_lat = []
list_lat_true = []

k = 0
for i in range(0,len(train_geolat)): #len(train_geolat)
    phototags = train_geolat.loc[i,'photoTags']    
    try:
        start = phototags.find('geolat')
        lat = int(phototags[start+6:].split()[0])
        start = phototags.find('geolon')
        lon = int(phototags[start+6:].split()[0])
        list_lat.append(lat)
        list_ln.append(lon)
        list_ln_true.append(float(train_geolat.loc[i,'longitude'] ))
        list_lat_true.append(float(train_geolat.loc[i,'latitude']))
    except: 
        k += 1
percent = k/i
print(percent)

367567
0.06609425246078256
